In [1]:
import pandas as pd
import gensim
from gensim.models import Word2Vec, KeyedVectors
import nltk
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Todd
[nltk_data]     Howard\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from IPython.display import clear_output

def do_stuff(embeds_path, df_path):

    df = pd.read_csv(df_path, sep=';')
    sentences = list(map(nltk.word_tokenize, map(lambda x: x.lower(), df['text_content'].values)))
    # sentences[6][5:10]

    model = KeyedVectors.load_word2vec_format(embeds_path, binary=False)

    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit([[d] for d in df['category']])
    y = enc.transform([[d] for d in df['category']]).toarray()

    X = []

    for i in range(len(sentences)):
        words = sentences[i]

        words_embeds = list(map(lambda x: model[x] if x in model else np.zeros_like(model[0]), words))

        doc_embed = np.average(np.array(words_embeds), axis = 0)
        X.append(doc_embed )

    X = np.array(X)

    reports = []

    kf = KFold(n_splits=5)

    for i, (train_index, test_index) in enumerate(kf.split(X)):
        clf_kfold = MLPClassifier(random_state=1, max_iter=300).fit(X[train_index], y[train_index])
        reports.append(classification_report(
            y[test_index], clf_kfold.predict(X[test_index]), 
            target_names=enc.categories_[0], output_dict = True))
        
    total_result = dict()

    for k in reports[0].keys():
        vals = [dc[k] for dc in reports]

        metrics_dict = dict()
        for k2 in vals[0].keys():
            vals2 = [dc[k2] for dc in vals]
            avg_val = (sum(vals2) * 1.) / len(vals2)
            metrics_dict[k2] = avg_val

        total_result[k] = metrics_dict

    clear_output(wait=True)
    
    return pd.DataFrame(total_result).transpose()


In [3]:
out = do_stuff('data/kgr10.plain.lemma.cbow.dim100.neg10.vec',  'data/final_recipes.csv')
out.to_csv('outs/fast_text_clarin.csv')
out

,precision,recall,f1-score,support
ciasta i torty,0.711383,0.643646,0.664700,771.4
ciasteczka,0.655591,0.525151,0.535185,456.0
desery,0.602146,0.561531,0.555538,788.8
grill,0.578675,0.434738,0.469566,147.8
inne,0.000000,0.000000,0.000000,97.8
napoje i koktajle,0.752804,0.553552,0.626673,137.4
obiady,0.672359,0.608628,0.631821,1137.8
pieczywo,0.296970,0.079266,0.125022,34.0
przekąski,0.468530,0.172222,0.247228,480.4
przetwory,0.707218,0.535242,0.600770,234.4
